# Purpose
* demo the common pandas method chaining decorated with scikit-lego

In [1]:
import pandas_pipe
import pandas as pd
import logging
import numpy as np
import pytest

In [2]:
stream_handler = logging.StreamHandler()
pandas_pipe.logger.addHandler(stream_handler)

## Explode example
* see the discussion and other implementations from the question [Split (explode) pandas dataframe string entry to separate rows](https://stackoverflow.com/questions/12680754/split-explode-pandas-dataframe-string-entry-to-separate-rows)  

In [3]:
dataf_input = pd.DataFrame(
    {
        "aaa": {0: 10, 1: 11, 2: 12, 3: 13},
        "myid": {0: 1, 1: 2, 2: 3, 3: 4},
        "num": {0: "1, 2, 3", 1: np.nan, 2: "1, 2", 3: np.nan},
        "text": {0: "aa, bb, cc", 1: np.nan, 2: "cc, dd", 3: "ee"},
        "states": {
            0: "Wyoming; Illinois; New Hampshire",
            1: "Pennsylvania",
            2: "New York",
            3: "Pennsylvania",
        },
    }
)
dataf_input

,aaa,myid,num,text,states
0,10,1,"1, 2, 3","aa, bb, cc",Wyoming; Illinois; New Hampshire
1,11,2,NaN,NaN,Pennsylvania
2,12,3,"1, 2","cc, dd",New York
3,13,4,NaN,ee,Pennsylvania


In [4]:
(
    dataf_input.pipe(pandas_pipe.start_pipeline)
    .pipe(pandas_pipe.explode_setup, columns=["num", "text"], delimiter=",")
    .pipe(pandas_pipe.explode_setup, columns=["states"], delimiter=";")
    .pipe(pandas_pipe.explode, column="num", ignore_index=False)
    .pipe(pandas_pipe.explode, column="text", ignore_index=False)
    .pipe(pandas_pipe.explode, column="states", ignore_index=False)
    .pipe(pandas_pipe.drop_duplicates)
)

[start_pipeline(df)] time=0:00:00.000093 n_obs=4, n_col=5 names=['aaa', 'myid', 'num', 'text', 'states']
[explode_setup(df, columns = ['num', 'text'], delimiter = ',')] time=0:00:00.000919 n_obs=4, n_col=5 delta=(0, 0)
[explode_setup(df, columns = ['states'], delimiter = ';')] time=0:00:00.000486 n_obs=4, n_col=5 delta=(0, 0)
[explode(df, column = 'num', ignore_index = False)] time=0:00:00.003621 n_obs=7, n_col=5 delta=(+3, 0)
[explode(df, column = 'text', ignore_index = False)] time=0:00:00.002656 n_obs=15, n_col=5 delta=(+8, 0)
[explode(df, column = 'states', ignore_index = False)] time=0:00:00.002139 n_obs=33, n_col=5 delta=(+18, 0)
[drop_duplicates(df)] time=0:00:00.001477 n_obs=33, n_col=5 delta=(0, 0)


,aaa,myid,num,text,states
0,10,1,1,aa,Wyoming
0,10,1,1,aa,Illinois
0,10,1,1,aa,New Hampshire
0,10,1,1,bb,Wyoming
0,10,1,1,bb,Illinois
0,10,1,1,bb,New Hampshire
0,10,1,1,cc,Wyoming
0,10,1,1,cc,Illinois
0,10,1,1,cc,New Hampshire
0,10,1,2,aa,Wyoming


## Explode setup
* the `explode` method takes [each element of a list-like to a row](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.explode.html)
* Therefore you may need to transform string delimited cells into a list

In [5]:
(
    dataf_input.pipe(pandas_pipe.start_pipeline)
    .pipe(pandas_pipe.explode_setup, columns=["num", "text"], delimiter=",")
    .pipe(pandas_pipe.explode_setup, columns=["states"], delimiter=";")
    .pipe(pandas_pipe.explode, column="num", ignore_index=True)
    .pipe(pandas_pipe.explode, column="text", ignore_index=True)
    .pipe(pandas_pipe.explode, column="states", ignore_index=True)
    .pipe(pandas_pipe.drop_duplicates)
)

[start_pipeline(df)] time=0:00:00.000102 n_obs=4, n_col=5 names=['aaa', 'myid', 'num', 'text', 'states']
[explode_setup(df, columns = ['num', 'text'], delimiter = ',')] time=0:00:00.000735 n_obs=4, n_col=5 delta=(0, 0)
[explode_setup(df, columns = ['states'], delimiter = ';')] time=0:00:00.000498 n_obs=4, n_col=5 delta=(0, 0)
[explode(df, column = 'num', ignore_index = True)] time=0:00:00.002588 n_obs=7, n_col=5 delta=(+3, 0)
[explode(df, column = 'text', ignore_index = True)] time=0:00:00.002324 n_obs=15, n_col=5 delta=(+8, 0)
[explode(df, column = 'states', ignore_index = True)] time=0:00:00.002150 n_obs=33, n_col=5 delta=(+18, 0)
[drop_duplicates(df)] time=0:00:00.001306 n_obs=33, n_col=5 delta=(0, 0)


,aaa,myid,num,text,states
0,10,1,1,aa,Wyoming
1,10,1,1,aa,Illinois
2,10,1,1,aa,New Hampshire
3,10,1,1,bb,Wyoming
4,10,1,1,bb,Illinois
5,10,1,1,bb,New Hampshire
6,10,1,1,cc,Wyoming
7,10,1,1,cc,Illinois
8,10,1,1,cc,New Hampshire
9,10,1,2,aa,Wyoming


## Be careful with using drop duplicates after setup
* `explode_setup` can apply to more than one column but don't forget to explode the column
* drop duplicates cannot drop list-like cells


In [6]:
with pytest.raises(TypeError):
    pandas_pipe.logger.error("TypeError: unhashable type: 'list'")
    (
        dataf_input.pipe(pandas_pipe.start_pipeline)
        .pipe(pandas_pipe.explode_setup, columns=["num", "text"], delimiter=",")
        .pipe(pandas_pipe.explode_setup, columns=["states"], delimiter=";")
        .pipe(pandas_pipe.explode, column="num", ignore_index=True)
        .pipe(pandas_pipe.drop_duplicates)
        .pipe(pandas_pipe.explode, column="text", ignore_index=True)
        .pipe(pandas_pipe.explode, column="states", ignore_index=True)
    )

TypeError: unhashable type: 'list'
[start_pipeline(df)] time=0:00:00.000079 n_obs=4, n_col=5 names=['aaa', 'myid', 'num', 'text', 'states']
[explode_setup(df, columns = ['num', 'text'], delimiter = ',')] time=0:00:00.000891 n_obs=4, n_col=5 delta=(0, 0)
[explode_setup(df, columns = ['states'], delimiter = ';')] time=0:00:00.000348 n_obs=4, n_col=5 delta=(0, 0)
[explode(df, column = 'num', ignore_index = True)] time=0:00:00.002762 n_obs=7, n_col=5 delta=(+3, 0)


## Explode only does one col at a time
* see more examples for the [answer](https://stackoverflow.com/a/57122617) to the question of [Split (explode) pandas dataframe string entry to separate rows](https://scikit-lego.readthedocs.io/en/latest/pandas_pipeline.html)

In [7]:
with pytest.raises(ValueError):
    pandas_pipe.logger.error("ValueError: column must be a scalar")
    (
        dataf_input.pipe(pandas_pipe.start_pipeline)
        .pipe(pandas_pipe.explode_setup, columns=["num", "text"], delimiter=",")
        .pipe(pandas_pipe.explode_setup, columns=["states"], delimiter=";")
        .pipe(pandas_pipe.explode, column=["num", "text", "states"], ignore_index=True)
        .pipe(pandas_pipe.drop_duplicates)
    )

ValueError: column must be a scalar
[start_pipeline(df)] time=0:00:00.000117 n_obs=4, n_col=5 names=['aaa', 'myid', 'num', 'text', 'states']
[explode_setup(df, columns = ['num', 'text'], delimiter = ',')] time=0:00:00.001133 n_obs=4, n_col=5 delta=(0, 0)
[explode_setup(df, columns = ['states'], delimiter = ';')] time=0:00:00.000549 n_obs=4, n_col=5 delta=(0, 0)


## Exploding columns may need `drop_duplicates` after explode
* Taken from the [answer](https://stackoverflow.com/a/62268744/3587374) to the question [Amount of rows that contain a specific word in a dataframe](https://stackoverflow.com/questions/62268523/amount-of-rows-that-contain-a-specific-word-in-a-dataframe)
* with `drop_duplicates` you count the frequency of words per document
* without `drop_duplicates` you count the frequency of each word among all documents

In [8]:
df = pd.DataFrame([{'customer': 'A', 'message': 'hi i need help i want a card'}, {'customer': 'B', 'message': 'i want a card'}])
df

,customer,message
0,A,hi i need help i want a card
1,B,i want a card


In [9]:
(
    df.pipe(pandas_pipe.start_pipeline)
    .pipe(pandas_pipe.explode_setup, columns=["message"], delimiter=" ")
    .pipe(pandas_pipe.explode, column="message")
    .pipe(pandas_pipe.drop_duplicates)
)["message"].value_counts()

[start_pipeline(df)] time=0:00:00.000119 n_obs=2, n_col=2 names=['customer', 'message']
[explode_setup(df, columns = ['message'], delimiter = ' ')] time=0:00:00.000946 n_obs=2, n_col=2 delta=(0, 0)
[explode(df, column = 'message')] time=0:00:00.004479 n_obs=12, n_col=2 delta=(+10, 0)
[drop_duplicates(df)] time=0:00:00.001075 n_obs=11, n_col=2 delta=(-1, 0)


i       2
card    2
a       2
want    2
hi      1
help    1
need    1
Name: message, dtype: int64

In [10]:
(
    df.pipe(pandas_pipe.start_pipeline)
    .pipe(pandas_pipe.explode_setup, columns=["message"], delimiter=" ")
    .pipe(pandas_pipe.explode, column="message")
)["message"].value_counts()

[start_pipeline(df)] time=0:00:00.000089 n_obs=2, n_col=2 names=['customer', 'message']
[explode_setup(df, columns = ['message'], delimiter = ' ')] time=0:00:00.000599 n_obs=2, n_col=2 delta=(0, 0)
[explode(df, column = 'message')] time=0:00:00.002005 n_obs=12, n_col=2 delta=(+10, 0)


i       3
card    2
a       2
want    2
hi      1
help    1
need    1
Name: message, dtype: int64

## With larger data will need to `drop_duplicates` after each explode
* See the [comments](https://stackoverflow.com/questions/60152241/pandas-explode-and-drop-duplicates-for-multiple-columns#comment106391902_60152732) from the question [Pandas explode and drop duplicates for multiple columns](https://stackoverflow.com/questions/60152241/pandas-explode-and-drop-duplicates-for-multiple-columns#comment106391902_60152732) excerpt below:
> *... This will generate duplicate rows (both on your and my solution), that need to be dropped intermediately after each .explode() otherwise the size of the dataframe becomes unmanageable ... Using drop_duplicates() after each explode() ensures the dataframe maintains a healthy size*

## Map column example

In [11]:
df = pd.DataFrame([{'customer': 'A', 'message': 'hi i need help i want a card'}, {'customer': 'B', 'message': 'i want a card'}])
df

,customer,message
0,A,hi i need help i want a card
1,B,i want a card


### __Note__
* the in place methods may *show* the `None` but is editing the cell
* see the question [pandas apply updates inplace but returns None](https://stackoverflow.com/questions/51310263/pandas-apply-updates-inplace-but-returns-none) for another example

In [12]:
df["message"].str.split().map(set).map(lambda x: x.remove("a"))

0    None
1    None
Name: message, dtype: object

### Separate the two steps 
1. creating the `set` within the cell 
2. applying the `remove`
3. Notice the `None` but the set was modified

In [13]:
df["message"] = df["message"].str.split().map(set)
df

,customer,message
0,A,"{hi, need, card, help, i, a, want}"
1,B,"{a, i, want, card}"


In [14]:
df["message"].map(lambda x: x.remove("a"))

0    None
1    None
Name: message, dtype: object

In [15]:
df

,customer,message
0,A,"{hi, need, card, help, i, want}"
1,B,"{i, want, card}"


* within the pipeline using the in place `remove` method passes `None` down the pipeline

In [16]:
df = pd.DataFrame([{'customer': 'A', 'message': 'hi i need help i want a card'}, {'customer': 'B', 'message': 'i want a card'}])
df

,customer,message
0,A,hi i need help i want a card
1,B,i want a card


In [17]:
(
    df.pipe(pandas_pipe.start_pipeline)
    .pipe(pandas_pipe.vectorize_str, ["message"], "split")
    .pipe(pandas_pipe.map_column, "message", set)
    .pipe(pandas_pipe.map_column, "message", lambda x: x.remove("a") if "a" in x else x)
)

[start_pipeline(df)] time=0:00:00.000072 n_obs=2, n_col=2 names=['customer', 'message']
[vectorize_str(df, columns = ['message'], method = 'split')] time=0:00:00.000547 n_obs=2, n_col=2 delta=(0, 0)
[map_column(df, column = 'message', arg = <class 'set'>)] time=0:00:00.000319 n_obs=2, n_col=2 delta=(0, 0)
[map_column(df, column = 'message', arg = <function <lambda> at 0x119996f28>)] time=0:00:00.000307 n_obs=2, n_col=2 delta=(0, 0)


,customer,message
0,A,None
1,B,None


* use a helper function to avoid passing None within the pipe

In [18]:
def remove(x, element):
    x.remove(element)
    return x

In [19]:
(
    df.pipe(pandas_pipe.start_pipeline)
    .pipe(pandas_pipe.vectorize_str, ["message"], "split")
    .pipe(pandas_pipe.map_column, "message", set)
    .pipe(pandas_pipe.map_column, "message", lambda x: remove(x, "a") if "a" in x else x)
)

[start_pipeline(df)] time=0:00:00.000103 n_obs=2, n_col=2 names=['customer', 'message']
[vectorize_str(df, columns = ['message'], method = 'split')] time=0:00:00.000594 n_obs=2, n_col=2 delta=(0, 0)
[map_column(df, column = 'message', arg = <class 'set'>)] time=0:00:00.000519 n_obs=2, n_col=2 delta=(0, 0)
[map_column(df, column = 'message', arg = <function <lambda> at 0x1199a2620>)] time=0:00:00.000430 n_obs=2, n_col=2 delta=(0, 0)


,customer,message
0,A,"{hi, need, card, help, i, want}"
1,B,"{i, want, card}"


In [20]:
pd.DataFrame([{'customer': 'A', 'message': {"card", "i", "want", "help", "need", "hi"}}, {'customer': 'B', 'message': {"card", "want", "i",}}])

,customer,message
0,A,"{hi, need, card, help, i, want}"
1,B,"{i, want, card}"


In [21]:
(
    df.pipe(pandas_pipe.start_pipeline)
    .pipe(pandas_pipe.vectorize_str, ["message"], "split")
    .pipe(pandas_pipe.map_column, "message", set, to_column="message_set")
    .pipe(pandas_pipe.map_column, "message_set", lambda x: remove(x, "a") if "a" in x else x)
)

[start_pipeline(df)] time=0:00:00.000086 n_obs=2, n_col=2 names=['customer', 'message']
[vectorize_str(df, columns = ['message'], method = 'split')] time=0:00:00.001745 n_obs=2, n_col=2 delta=(0, 0)
[map_column(df, column = 'message', arg = <class 'set'>, to_column = 'message_set')] time=0:00:00.001095 n_obs=2, n_col=3 delta=(0, +1)
[map_column(df, column = 'message_set', arg = <function <lambda> at 0x1199a2a60>)] time=0:00:00.000550 n_obs=2, n_col=3 delta=(0, 0)


,customer,message,message_set
0,A,"[hi, i, need, help, i, want, a, card]","{hi, need, card, help, i, want}"
1,B,"[i, want, a, card]","{i, want, card}"


## `query` drop and `astype`

* data taken from the question [Deleting DataFrame row in Pandas based on column value](https://stackoverflow.com/questions/18172851/deleting-dataframe-row-in-pandas-based-on-column-value)
* query drop is adopted within the [answer](https://stackoverflow.com/a/18173088) 

In [22]:
df = pd.DataFrame(
    {
        "line_date": {
            0: "2007-03-31",
            1: "2007-03-10",
            2: "2007-02-10",
            3: "2007-01-13",
            4: "2006-12-23",
            5: "2006-11-09",
            6: "2006-10-22",
            7: "2006-09-29",
            8: "2006-09-16",
            9: "2006-08-30",
            10: "2006-02-11",
            11: "2006-01-13",
            12: "2006-01-02",
            13: "2005-12-06",
        },
        "daysago": {
            0: 62,
            1: 83,
            2: 111,
            3: 139,
            4: 160,
            5: 204,
            6: 222,
            7: 245,
            8: 258,
            9: 275,
            10: 475,
            11: 504,
            12: 515,
            13: 542,
        },
        "line_race": {
            0: 11,
            1: 11,
            2: 9,
            3: 10,
            4: 10,
            5: 9,
            6: 8,
            7: 9,
            8: 11,
            9: 8,
            10: 5,
            11: 0,
            12: 0,
            13: 0,
        },
        "rating": {
            0: 56,
            1: 67,
            2: 66,
            3: 83,
            4: 88,
            5: 52,
            6: 66,
            7: 70,
            8: 68,
            9: 72,
            10: 65,
            11: 70,
            12: 64,
            13: 70,
        },
        "rw": {
            0: 1.0,
            1: 1.0,
            2: 1.0,
            3: 0.880678,
            4: 0.793033,
            5: 0.636655,
            6: 0.581946,
            7: 0.518825,
            8: 0.486226,
            9: 0.446667,
            10: 0.164591,
            11: 0.142409,
            12: 0.1348,
            13: 0.117803,
        },
        "wrating": {
            0: 56.0,
            1: 67.0,
            2: 66.0,
            3: 73.096278,
            4: 69.786942,
            5: 33.106077,
            6: 38.408408,
            7: 36.317752,
            8: 33.063381,
            9: 32.160051,
            10: 10.698423,
            11: 9.968634,
            12: 8.627219,
            13: 8.246238,
        },
    }
)

In [23]:
df

,line_date,daysago,line_race,rating,rw,wrating
0,2007-03-31,62,11,56,1.000000,56.000000
1,2007-03-10,83,11,67,1.000000,67.000000
2,2007-02-10,111,9,66,1.000000,66.000000
3,2007-01-13,139,10,83,0.880678,73.096278
4,2006-12-23,160,10,88,0.793033,69.786942
5,2006-11-09,204,9,52,0.636655,33.106077
6,2006-10-22,222,8,66,0.581946,38.408408
7,2006-09-29,245,9,70,0.518825,36.317752
8,2006-09-16,258,11,68,0.486226,33.063381
9,2006-08-30,275,8,72,0.446667,32.160051


In [24]:
(
    df.pipe(pandas_pipe.start_pipeline)
    .pipe(pandas_pipe.astype, {'line_date': 'datetime64'})
    .pipe(pandas_pipe.query, 'line_race != 0')
).dtypes

[start_pipeline(df)] time=0:00:00.000095 n_obs=14, n_col=6 names=['line_date', 'daysago', 'line_race', 'rating', 'rw', 'wrating']
[astype(df, dtype = {'line_date': 'datetime64'})] time=0:00:00.002192 n_obs=14, n_col=6 delta=(0, 0)
[query(df, expr = 'line_race != 0')] time=0:00:00.009289 n_obs=11, n_col=6 delta=(-3, 0)


line_date    datetime64[ns]
daysago               int64
line_race             int64
rating                int64
rw                  float64
wrating             float64
dtype: object

## Replace

In [25]:
df = pd.DataFrame({'A': ['bat', 'foo', 'bait'],
                   'B': ['abc', 'bar', 'xyz']})
df

,A,B
0,bat,abc
1,foo,bar
2,bait,xyz


In [26]:
(
    df.pipe(pandas_pipe.start_pipeline)
    .pipe(pandas_pipe.replace, to_replace={'A': r'^ba.$'} , value='new', regex=True)
)

[start_pipeline(df)] time=0:00:00.000133 n_obs=3, n_col=2 names=['A', 'B']
[replace(df, kwargs = {'to_replace': {'A': '^ba.$'}, 'value': 'new', 'regex': True})] time=0:00:00.001391 n_obs=3, n_col=2 delta=(0, 0)


,A,B
0,new,abc
1,foo,bar
2,bait,xyz
